In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
PstBook = xl.load_workbook('Pst_IRS.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CCY = 'TWD'
Type = 'IRS'
MTMDate = MktSheet.cell(1, 3).value

In [4]:
print(CCY)
print(Type)
print(MTMDate)

TWD
IRS
2021-06-30 00:00:00


In [5]:
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
DC365 = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [6]:
ZeroSP = MktSheet.cell(43, 5).value
Zero3M = MktSheet.cell(44, 5).value
Zero6M = MktSheet.cell(45, 5).value
Zero1Y = MktSheet.cell(46, 5).value
Zero2Y = MktSheet.cell(47, 5).value
Zero3Y = MktSheet.cell(48, 5).value
Zero5Y = MktSheet.cell(49, 5).value
Zero10Y = MktSheet.cell(50, 5).value
Zero15Y = MktSheet.cell(51, 5).value
Zero20Y = MktSheet.cell(52, 5).value
Zero30Y = MktSheet.cell(53, 5).value
print(ZeroSP)

0.004795999997252601


In [7]:
DateSP = evDate
Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
Date2Y = Cal.advance(evDate, 2, ql.Years, BDC, False)
Date3Y = Cal.advance(evDate, 3, ql.Years, BDC, False)
Date5Y = Cal.advance(evDate, 5, ql.Years, BDC, False)
Date10Y = Cal.advance(evDate, 10, ql.Years, BDC, False)
Date15Y = Cal.advance(evDate, 15, ql.Years, BDC, False)
Date20Y = Cal.advance(evDate, 20, ql.Years, BDC, False)
Date30Y = Cal.advance(evDate, 30, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [8]:
ListDate = list([DateSP, Date3M, Date6M, Date1Y, Date2Y, Date3Y, Date5Y, Date10Y, Date15Y, Date20Y, Date30Y])
ListRate = list([ZeroSP, Zero3M, Zero6M, Zero1Y, Zero2Y, Zero3Y, Zero5Y, Zero10Y, Zero15Y, Zero20Y, Zero30Y])
VecDate = ql.DateVector(ListDate)
VecRate = ql.DoubleVector(ListRate)
TSZero = ql.ZeroCurve(VecDate, VecRate, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)

In [9]:
IRSSheet = PstBook['IRS']
start_row = 2
end_row = 4
start_col = 0
end_col = 7
recordcount = end_col - start_col + 1
ListIRS = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in IRSSheet[i][start_col:end_col+1]]
    ListIRS.append(row)
    print(row)

['IRS001', datetime.datetime(2021, 2, 19, 0, 0), datetime.datetime(2021, 2, 21, 0, 0), datetime.datetime(2024, 2, 21, 0, 0), 'Long', 0.006, 0, 100000000]
['IRS002', datetime.datetime(2021, 4, 16, 0, 0), datetime.datetime(2021, 4, 18, 0, 0), datetime.datetime(2023, 4, 18, 0, 0), 'Short', 0.0053, 0, 100000000]
['IRS003', datetime.datetime(2021, 6, 21, 0, 0), datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2025, 6, 23, 0, 0), 'Long', 0.0065, 0, 100000000]


In [10]:
IRSBook = list()
for i in range(len(ListIRS)):
    row = ListIRS[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[2]))
    record.append(DatetimeToDate(row[3]))
    record.append(row[4])
    record.append(row[5])
    record.append(row[6])
    record.append(row[7])
    IRSBook.append(record)

In [11]:
IRSBook

[['IRS001', Date(21,2,2021), Date(21,2,2024), 'Long', 0.006, 0, 100000000],
 ['IRS002', Date(18,4,2021), Date(18,4,2023), 'Short', 0.0053, 0, 100000000],
 ['IRS003', Date(23,6,2021), Date(23,6,2025), 'Long', 0.0065, 0, 100000000]]

In [12]:
h_TSZero = ql.YieldTermStructureHandle(TSZero)
IRSEngine = ql.DiscountingSwapEngine(h_TSZero)

In [13]:
ListZero = ListRate
VecDate = ql.DateVector(ListDate)
ListListZero = list()
ListListZero.append(ListZero)

In [14]:
ListZero

[0.004795999997252601,
 0.004795999997762682,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [15]:
NoCurve = len(ListZero)
print(NoCurve)
ListRW = list([0.017, 0.017, 0.016, 0.013, 0.012, 0.011, 0.011, 0.011, 0.011, 0.011])

11


In [16]:
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoCurve):
        if(i==j):
            NewList.append(ListZero[j] + 0.0001)
        else:
            NewList.append(ListZero[j])
    ListListZero.append(NewList)

In [17]:
ListListZero[0]

[0.004795999997252601,
 0.004795999997762682,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [18]:
ListListZero[1]

[0.004795999997252601,
 0.004895999997762682,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [19]:
ListListZero[2]

[0.004795999997252601,
 0.004795999997762682,
 0.005779705459626833,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [20]:
ListListZero[10]

[0.004795999997252601,
 0.004795999997762682,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008970403457750699]

In [21]:
ListVecZero = list()
ListTSZero = list()
for i in range(NoCurve):
    AVecZero = ql.DoubleVector(ListListZero[i])
    ListVecZero.append(AVecZero)
    ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
    ListTSZero.append(ATSZero)

In [22]:
CurvUpListListZero = list()
CurvUpListListZero.append(ListZero)
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoCurve):
        if(i==j):
            NewList.append(ListZero[j] + 0.017)
        else:
            NewList.append(ListZero[j])
    CurvUpListListZero.append(NewList)

In [23]:
CurvUpListListZero[0]

[0.004795999997252601,
 0.004795999997762682,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [24]:
CurvUpListListZero[1]

[0.004795999997252601,
 0.02179599999776268,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [25]:
CurvUpListListZero[2]

[0.004795999997252601,
 0.004795999997762682,
 0.022679705459626835,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [26]:
CurvUpListListZero[10]

[0.004795999997252601,
 0.004795999997762682,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.0258704034577507]

In [27]:
CurvDnListListZero = list()
CurvDnListListZero.append(ListZero)
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoCurve):
        if(i==j):
            NewList.append(ListZero[j] - 0.017)
        else:
            NewList.append(ListZero[j])
    CurvDnListListZero.append(NewList)

In [28]:
CurvDnListListZero[0]

[0.004795999997252601,
 0.004795999997762682,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [29]:
CurvDnListListZero[1]

[0.004795999997252601,
 -0.01220400000223732,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [30]:
P3M = ql.Period(3, ql.Months)
TWD = ql.TWDCurrency()
EOM = False
FixingDays = 2
TAIBOR3M = ql.IborIndex("TAIBOR", P3M, FixingDays, TWD, Cal, BDC, EOM, DC365, h_TSZero)

In [31]:
FixingDates = list()
FixingRates = list()
for i in range(33, 131):
    FixDate = DatetimeToDate(MktSheet.cell(i, 7).value)
    FixRate = MktSheet.cell(i, 8).value / 100.0
    FixingDates.append(FixDate)
    FixingRates.append(FixRate)
VecFixingDates = ql.DateVector(FixingDates)
VecFixingRates = ql.DoubleVector(FixingRates)    
TAIBOR3M.addFixings(VecFixingDates, VecFixingRates, True)

In [32]:
SenSheet = PstBook['GIRR']
for i in range(0, len(IRSBook)):
    ListValue = list()

    record = IRSBook[i]
    valueDate = record[1] # Effective
    maturityDate = record[2] #Maturity
    if record[3] == 'Long':
        LS = ql.VanillaSwap.Payer
    else:
        LS = ql.VanillaSwap.Receiver
        
    fixedRate = record[4]
    floatSpread = record[5]
    NPA = record[6]
    DGR = ql.DateGeneration.Backward
    sch = ql.Schedule(valueDate, maturityDate, P3M, Cal, BDC, BDC, DGR, False)        
    IRS = ql.VanillaSwap(LS, NPA, sch, fixedRate, DC365, sch, TAIBOR3M, floatSpread, DC365)
    
    h_TS = ql.YieldTermStructureHandle(ListTSZero[0])
    IRSEngine = ql.DiscountingSwapEngine(h_TS)    
    IRS.setPricingEngine(IRSEngine)
    
    Value = IRS.NPV()      
    ListValue.append(Value)
    SenSheet.cell(i+5, 1).value = record[0]
    SenSheet.cell(i+5, 2).value = ListValue[0]
    
    for j in range(1, NoCurve):
        h_TS = ql.YieldTermStructureHandle(ListTSZero[j])
        IRSEngine = ql.DiscountingSwapEngine(h_TS)
        IRS.setPricingEngine(IRSEngine)
        Value = IRS.NPV()
        ListValue.append(Value)
    
    ListSensitivity = list()
    for j in range(1, len(ListValue)):
        ListSensitivity.append((ListValue[j] - ListValue[0])*10000)
    
    ListWS = list()
    for j in range(0, len(ListSensitivity)):
        ListWS.append(ListRW[j] * ListSensitivity[j])
                
    for j in range(0, len(ListSensitivity)):
        SenSheet.cell(i+5, j+5).value = ListWS[j]
    
    ListVecZero = list()
    CurvUpListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvUpListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Simple, ql.Annual)
        CurvUpListTSZero.append(ATSZero)
                
    ListVecZero = list()
    CurvDnListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvDnListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Simple, ql.Annual)
        CurvDnListTSZero.append(ATSZero)
        
    CurvUpListHTSZero = list()
    for j in range(NoCurve):
        CurvUpListHTSZero.append(ql.YieldTermStructureHandle(CurvUpListTSZero[j]))
        
    CurvDnListHTSZero = list()
    for j in range(NoCurve):
        CurvDnListHTSZero.append(ql.YieldTermStructureHandle(CurvDnListTSZero[j]))
        
    CurvUpValue = list()
    for j in range(NoCurve):
        IRSEngine = ql.DiscountingSwapEngine(CurvUpListHTSZero[j])
        IRS.setPricingEngine(IRSEngine)
        Value = IRS.NPV()
        CurvUpValue.append(Value)
    
    CurvDnValue = list()
    for j in range(NoCurve):
        IRSEngine = ql.DiscountingSwapEngine(CurvDnListHTSZero[j])
        IRS.setPricingEngine(IRSEngine)
        Value = IRS.NPV()
        CurvDnValue.append(Value)        
        
    CurvatureUp = list()
    ShiftUpValue = list()
    DeltaChange = list()
    for j in range(1, NoCurve):
        ShiftUpValue.append(CurvUpValue[j]-CurvUpValue[0])        
        CurvatureUp.append(-((CurvUpValue[j]-CurvUpValue[0]) - 0.017 * ListSensitivity[j-1]))
                
    for j in range(0, len(CurvatureUp)):    
        SenSheet.cell(i+5, j+17).value = CurvatureUp[j]
            
    CurvatureDn = list()
    ShiftDnValue = list()
    for j in range(1, NoCurve):
        ShiftDnValue.append(CurvDnValue[j]-CurvDnValue[0])    
        CurvatureDn.append(-((CurvDnValue[j]-CurvDnValue[0]) + 0.017 * ListSensitivity[j-1]))

    for j in range(0, len(CurvatureDn)):    
        SenSheet.cell(i+5, j+29).value = CurvatureDn[j]
        
PstBook.save('Pst_IRS.xlsx')
PstBook.close()

In [33]:
ListValue

[-51846.699520635884,
 -51845.76624800172,
 -51844.85928159999,
 -51834.73356262222,
 -51842.11996093346,
 -51887.065469846595,
 -51862.25760634476,
 -51846.699520635884,
 -51846.699520635884,
 -51846.699520635884,
 -51846.699520635884]

In [34]:
ListSensitivity

[9332.726341672242,
 18402.39035896957,
 119659.58013664931,
 45795.59702426195,
 -403659.49210710824,
 -155580.85708878934,
 0.0,
 0.0,
 0.0,
 0.0]

In [35]:
ListRW

[0.017, 0.017, 0.016, 0.013, 0.012, 0.011, 0.011, 0.011, 0.011, 0.011]

In [36]:
ListWS

[158.6563478084281,
 312.8406361024827,
 1914.553282186389,
 595.3427613154054,
 -4843.913905285299,
 -1711.3894279766828,
 0.0,
 0.0,
 0.0,
 0.0]